# Import Lib

In [1]:
!pip install -q tflite-model-maker

import numpy as np
import os

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import question_answer
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.question_answer import DataLoader
from tflite_model_maker.config import QuantizationConfig

     |████████████████████████████████| 593kB 2.8MB/s 
     |████████████████████████████████| 1.2MB 16.3MB/s 
     |████████████████████████████████| 112kB 20.2MB/s 
     |████████████████████████████████| 122kB 15.7MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 686kB 19.8MB/s 
     |████████████████████████████████| 645kB 19.8MB/s 
     |████████████████████████████████| 6.3MB 19.3MB/s 
     |████████████████████████████████| 174kB 52.1MB/s 
     |████████████████████████████████| 849kB 53.6MB/s 
     |████████████████████████████████| 92kB 12.2MB/s 
     |████████████████████████████████| 1.1MB 50.1MB/s 
     |████████████████████████████████| 38.2MB 76kB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 358kB 42.9MB/s 
     |████████████████████████████████| 194kB 55.4MB/s 


# Load Model and Data

In [2]:
spec = model_spec.get('mobilebert_qa')

In [3]:
# Follow TF Lite Bert Tutorial as reference and dataset
train_data_path = tf.keras.utils.get_file(
    fname='triviaqa-web-train-8000.json',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/tflite/dataset/triviaqa-web-train-8000.json')
validation_data_path = tf.keras.utils.get_file(
    fname='triviaqa-verified-web-dev.json',
    origin='https://storage.googleapis.com/download.tensorflow.org/models/tflite/dataset/triviaqa-verified-web-dev.json')

1171456/1167744 [==============================] - 0s 0us/step


In [4]:
train_data = DataLoader.from_squad(train_data_path, spec, is_training=True)
validation_data = DataLoader.from_squad(validation_data_path, spec, is_training=False)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


# Transfer Learning

In [5]:
model = question_answer.create(train_data, model_spec=spec, epochs=10)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/10
1067/1067 [==============================] - 720s 632ms/step - loss: 2.1951 - start_positions_loss: 2.2233 - end_positions_loss: 2.1669
Epoch 2/10
1067/1067 [==============================] - 675s 632ms/step - loss: 0.9977 - start_positions_loss: 0.9951 - end_positions_loss: 1.0003
Epoch 3/10
1067/1067 [==============================] - 675s 632ms/step - loss: 0.6215 - start_positions_loss: 0.6201 - end_positions_loss: 0.6230
Epoch 4/10
1067/1067 [==============================] - 675s 632ms/step - loss: 0.3371 - start_positions_loss: 0.3370 - end_positions_loss: 0.3372
Epoch 5/10
1067/1067 [==============================] - 675s 633ms/step - loss: 0.1795 - start_positions_loss: 0.1802 - end_positions_loss: 0.1788
Epoch 6/10
1067/1067 [==============================] - 676s 633ms/step - loss: 0.0944 - start_positions_loss: 0.0948 - end_positions_loss: 0.0940
Epoch 7/10
1067/1067 [==============================] - 675s 633ms/step - loss: 0.0476 - start_positions_loss: 0.0481 

In [6]:
model.summary()

Model: "bert_span_labeler"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 384)]        0                                            
__________________________________________________________________________________________________
core_model (Functional)         [(None, 384, 512), ( 24581888    input_word_ids[0][0]             
                                                                 input_mask[0][0] 

# Model Evaluation

In [7]:
model.evaluate(validation_data)

INFO:tensorflow:Made predictions for 200 records.


INFO:tensorflow:Made predictions for 200 records.


INFO:tensorflow:Made predictions for 400 records.


INFO:tensorflow:Made predictions for 400 records.


INFO:tensorflow:Made predictions for 600 records.


...

INFO:tensorflow:Made predictions for 1000 records.


INFO:tensorflow:Made predictions for 1000 records.


INFO:tensorflow:Made predictions for 1200 records.


INFO:tensorflow:Made predictions for 1200 records.


{'exact_match': 0.5136054421768708, 'final_f1': 0.6083441313033151}

# Export Model 

In [14]:
config = QuantizationConfig.for_float16()

In [15]:
model.export(export_dir='.', export_format=ExportFormat.TFLITE, quantization_config=config)

INFO:tensorflow:Assets written to: /tmp/tmpy3gu4tmy/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpy3gu4tmy/saved_model/assets


INFO:tensorflow:Vocab file is inside the TFLite model with metadata.


INFO:tensorflow:Vocab file is inside the TFLite model with metadata.


INFO:tensorflow:Saved vocabulary in /tmp/tmpoewucfkl/vocab.txt.


...

INFO:tensorflow:The associated file that has been been packed to the model is:


INFO:tensorflow:['vocab.txt']


INFO:tensorflow:['vocab.txt']


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


In [16]:
model.evaluate_tflite('model.tflite', validation_data)

INFO:tensorflow:Made predictions for 100 records.


INFO:tensorflow:Made predictions for 100 records.


INFO:tensorflow:Made predictions for 200 records.


INFO:tensorflow:Made predictions for 200 records.


INFO:tensorflow:Made predictions for 300 records.


...

INFO:tensorflow:Made predictions for 1100 records.


INFO:tensorflow:Made predictions for 1100 records.


INFO:tensorflow:Made predictions for 1200 records.


INFO:tensorflow:Made predictions for 1200 records.


{'exact_match': 0.5136054421768708, 'final_f1': 0.6083441313033151}